# PL05. Logistic Regression _Scratch_ (with Gradient Descent)

__Borja González Seoane. Machine Learning. Course 2024-25__



In this *notebook* we proceed to implement a logistic regression model from scratch (_scratch_), that is, without using Scikit-Learn libraries. The gradient descent algorithm will be implemented to minimize the cost function.

In this *notebook* a _dummy_ dataset will simply be used to test the model. The part of the exercise related to working with the Titanic dataset will be performed in the next *notebook* of PL05, to also contrast between different models.


In [ ]:
import numpy as np
from sklearn.datasets import (  # Interesting for generating test data
    make_classification,
)

## Creation of _dummy_ data

In [2]:
data = make_classification(
    n_samples = 1000,
    n_features = 20,
)

display(data)
x = data[0]
y = data[1]

(array([[-2.15017405,  0.78458206, -0.16330173, ...,  0.04469772,
          0.8137789 ,  1.03922619],
        [-0.93768773,  0.48820379,  1.64637422, ..., -0.81955796,
         -0.57678974, -0.69439466],
        [ 0.03517578, -0.36665391,  0.04038387, ..., -1.24407626,
         -0.41888296,  1.0266307 ],
        ...,
        [ 1.00983071,  0.43162932, -0.17442641, ...,  1.6352267 ,
         -0.90310221,  1.23752688],
        [ 0.71331689, -1.24903904,  1.15858324, ...,  0.32303203,
          0.75805069, -0.94071444],
        [-1.15180736, -0.68909235, -1.52811751, ..., -0.72661141,
         -1.2500449 , -0.63578947]]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
        1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
        1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
        1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
   

## Model implementation by parts

First we proceed to implement the necessary functions for the logistic regression model. The following functions will be implemented:

1. `predict`: $W^TX + b$, where $W$ is the weight vector and $b$ is the bias.
2. `sigmoid`: $\frac{1}{1 + e^{-\text{predict}(X)}}$.
3. `loss`: $-\frac{1}{m} \sum_{i=1}^{m} y_i \log(\text{sigmoid}(X_i)) + (1 - y_i) \log(1 - \text{sigmoid}(X_i))$.
4. `dl_dw`, derivative of the cost function with respect to weights: $\frac{1}{m} \sum_{i=1}^{m} (\text{sigmoid}(X_i) - y_i)X_i$.
5. `dl_db`, derivative of the cost function with respect to bias: $\frac{1}{m} \sum_{i=1}^{m} (\text{sigmoid}(X_i) - y_i)$.
6. `update`, weight and bias update: $W = W - \alpha \text{dl\_dw}$ and $b = b - \alpha \text{dl\_db}$, where $\alpha$ is the learning rate.
7. `fit`, function that, based on the previous pieces, trains the model for a given number of iterations.



In [ ]:
def predict(X, w, b):
    """
    Prediction: linear combination of weights and features.

    :param X: Input data.
    :param w: Weights.
    :param b: Bias.
    :return: Linear predictions.
    """
    return np.dot(X, w) + b

import numpy as np

def sigmoid(y_hat):
    """
    Sigmoid function to map values to probabilities in the range `[0, 1]`.

    :param y_hat: Linear prediction.
    :return: Values transformed to probabilities.
    """
    # Limit `y_hat` values to avoid overflow in the exponential
    y_hat = np.clip(y_hat, -500, 500)

    return 1 / (1 + np.exp(-y_hat))




def loss(y, sigmoid):
    """
    Logistic loss function.

    :param y: True labels.
    :param sigmoid: Probabilistic predictions.
    :return: Average loss (scalar).
    """
    # Limit `sigmoid` values to avoid calculation errors
    sigmoid = np.clip(sigmoid, 1e-15, 1 - 1e-15)

    return -(y * np.log(sigmoid) + (1 - y) * np.log(1 - sigmoid)).mean()


In [ ]:
def dldw(X, y, sigmoid):
    """
    Gradient of loss with respect to weights.

    :param X: Input data.
    :param y: True labels.
    :param sigmoid: Probabilistic predictions.
    :return: Weight gradients.
    """
    return np.dot(X.T, (sigmoid - y)) / X.shape[0]

def dldb(y, sigmoid):
    """
    Gradient of loss with respect to bias.

    :param y: True labels.
    :param sigmoid: Probabilistic predictions.
    :return: Bias gradient.
    """
    return (sigmoid - y).mean()

In [ ]:
def update(a, g, lr):
    """
    Update weights or bias value using gradient descent.

    :param a: Current value.
    :param g: Gradient.
    :param lr: Learning rate.
    :return: Updated value.
    """
    return a - (g * lr)


def fit(X, y, learning_rate=0.1, n_iter=100, monitor_loss=False):
    """
    Fit the logistic regression model.

    :param X: Input data.
    :param y: Class labels.
    :param learning_rate: Learning rate.
    :param n_iter: Number of iterations.
    :param monitor_loss: Whether to monitor loss.
    :return: Fitted weights and bias.
    """
    # Initialize weights based on the number of features
    w = np.zeros(X.shape[1])
    b = 0

    # Iterate the specified number of epochs
    for i in range(n_iter):
        # Initial prediction: linear combination of weights and features
        y_hat = predict(X, w, b)

        # Apply sigmoid function to get probabilities: map to `[0, 1]`
        sig = sigmoid(y_hat)

        # Calculate loss, in this case only for monitoring, if applicable
        if monitor_loss:
            loss_value = loss(y, sig)

            # Print loss value every 10 iterations
            if (i + 1) % 10 == 0:
                print(f"[Iter. {i+1}/{n_iter}] Loss = {loss_value}")

        # Calculate gradients of weights and bias
        grad_w = dldw(X, y, sig)
        grad_b = dldb(y, sig)

        # Update weights and bias by performing a gradient descent step
        w = update(w, grad_w, learning_rate)
        b = update(b, grad_b, learning_rate)

    return w, b



## Complete model archetype

Once the previous functions have been implemented and some simple numerical tests have been performed, we will proceed to implement the complete logistic regression model in the form of a class, with the `fit` and `predict` methods, thus following the usual archetype that has been used throughout the course.

In [ ]:
import numpy as np


class LogisticRegressionClassifierScratch:
    def __init__(self, learning_rate=0.1, n_iter=100, monitor_loss=False):
        self.learning_rate = learning_rate
        self.n_iter = n_iter
        self.b = 0
        self.w = None  # Will be reinitialized in the `fit` method

        self._monitor_loss = monitor_loss

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Fit the logistic regression model.

        :param X: Input data.
        :param y: Class labels.
        """
        # Initialize weights based on the number of features. Lazy initialization because
        # we need to know the number of features in the input data, which we don't know in the
        # constructor
        self.w = np.zeros(X.shape[1])

        # Iterate the specified number of epochs
        for i in range(self.n_iter):
            # Initial prediction: linear combination of weights and features
            y_hat = self.__predict(X, self.w, self.b)

            # Apply sigmoid function to get probabilities: map to `[0, 1]`
            sig = self.__sigmoid(y_hat)

            # Calculate loss, in this case only for monitoring, if applicable
            if self._monitor_loss:
                loss = self.__loss(y, sig)

                # Print loss value every 10 iterations
                if (i + 1) % 10 == 0:
                    print(f"[Iter. {i+1}/{self.n_iter}] Loss = {loss}")

            # Calculate gradients of weights and bias
            grad_w = self.__dldw(X, y, sig)
            grad_b = self.__dldb(y, sig)

            # Update weights and bias by performing a gradient descent step
            self.w = self.__update(self.w, grad_w, self.learning_rate)
            self.b = self.__update(self.b, grad_b, self.learning_rate)

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Predict class labels. Return predicted labels after applying
        a threshold of 0.5 to probabilities.

        :param X: Input data.
        :return: Predicted class labels.
        """
        # Initial prediction: linear combination of weights and features
        y_hat = self.__predict(X, self.w, self.b)

        # Apply sigmoid function to get probabilities: map to `[0, 1]`
        probs = self.__sigmoid(y_hat)

        # Threshold probabilities to get class labels, which would be 0 or 1
        return (probs >= 0.5).astype(int)

    ###############################################################
    # Include the functions developed in the previous cells as private methods
    # of the class. Static methods are used for convenience, since there's no need to access
    # class attributes, which are passed as arguments to the methods

    @staticmethod
    def __predict(X, w, b):
        """
        Prediction: linear combination of weights and features.

        :param X: Input data.
        :param w: Weights.
        :param b: Bias.
        :return: Linear predictions.
        """
        return np.dot(X, w) + b

    @staticmethod
    def __sigmoid(y_hat):
        """
        Sigmoid function to map values to probabilities in the range `[0, 1]`.

        :param y_hat: Linear prediction.
        :return: Values transformed to probabilities.
        """
        # Limit `y_hat` values to avoid overflow in the exponential
        y_hat = np.clip(y_hat, -500, 500)

        return 1 / (1 + np.exp(-y_hat))

    @staticmethod
    def __loss(y, sigmoid):
        """
        Logistic loss function.

        :param y: True labels.
        :param sigmoid: Probabilistic predictions.
        :return: Average loss (scalar).
        """
        # Limit `sigmoid` values to avoid calculation errors
        sigmoid = np.clip(sigmoid, 1e-15, 1 - 1e-15)

        return -(y * np.log(sigmoid) + (1 - y) * np.log(1 - sigmoid)).mean()

    @staticmethod
    def __dldw(X, y, sigmoid):
        """
        Gradient of loss with respect to weights.

        :param X: Input data.
        :param y: True labels.
        :param sigmoid: Probabilistic predictions.
        :return: Weight gradients.
        """
        return np.dot(X.T, (sigmoid - y)) / X.shape[0]

    @staticmethod
    def __dldb(y, sigmoid):
        """
        Gradient of loss with respect to bias.

        :param y: True labels.
        :param sigmoid: Probabilistic predictions.
        :return: Bias gradient.
        """
        return (sigmoid - y).mean()

    @staticmethod
    def __update(a, g, lr):
        """
        Update weights or bias value using gradient descent.

        :param a: Current value.
        :param g: Gradient.
        :param lr: Learning rate.
        :return: Updated value.
        """
        return a - (g * lr)
